In [1]:
!pip list

Package                                  Version
---------------------------------------- -----------
Adafruit-Blinka                          8.66.0
Adafruit-Blinka-Raspberry-Pi5-Neopixel   1.0.0rc2
adafruit-circuitpython-busdevice         5.2.13
adafruit-circuitpython-connectionmanager 3.1.5
adafruit-circuitpython-motor             3.4.17
adafruit-circuitpython-pca9685           3.4.19
adafruit-circuitpython-register          1.10.4
adafruit-circuitpython-requests          4.1.13
adafruit-circuitpython-servokit          1.3.21
adafruit-circuitpython-typing            1.12.2
Adafruit-PlatformDetect                  3.83.1
Adafruit-PureIO                          1.1.11
asttokens                                3.0.0
binho-host-adapter                       0.1.6
colorzero                                2.0
comm                                     0.2.3
debugpy                                  1.8.16
decorator                                5.2.1
executing                               

In [ ]:
!pip install opencv-python

In [1]:
import cv2
import time

# Open camera
cap = cv2.VideoCapture("/dev/video0")  # use /dev/video1 if needed
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Grab one frame
ret, frame = cap.read()
cap.release()

if not ret:
    print("❌ Failed to grab image")
    exit()


cv2.imshow("Captured", frame)
cv2.waitKey(2000)  # display for 2 seconds
cv2.destroyAllWindows()

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
avg_brightness = gray.mean()
print(avg_brightness)


139.71228841145833
